1. 데이터 로딩

저장된 CSV 파일에서 학습 및 테스트 데이터셋을 로드하는 것입니다. `pd.read_csv()` 함수는 CSV 파일을 읽고 Pandas DataFrame `train_prod_df` 및 `test_prod_df`로 로드합니다.

In [ ]:
# 1. Data Loading
train_prod_df = pd.read_csv('data/train.csv')
test_prod_df = pd.read_csv('data/test.csv')

2. 데이터 전처리


전처리 단계에는 여러 하위 단계가 포함됩니다.

쉼표 제거 및 데이터 타입 변경
test_prod_df 데이터프레임의 'employee1' 및 'employee2' 열은 문자열 타입에서 실수 타입으로 변환됩니다. 변환 전에 값에 포함된 쉼표를 제거합니다. train_prod_df 데이터프레임의 경우, 'employee1'과 'employee2'는 실수 타입으로 직접 변환됩니다. 추가로, train_prod_df의 'OC' 열에 있는 모든 공백이 제거됩니다.

In [ ]:
# 2. Data Preprocessing
# - Remove commas and change data types
test_prod_df.employee1 = test_prod_df.employee1.astype('str').str.replace(",", "").astype('float')
test_prod_df.employee2 = test_prod_df.employee2.astype('str').str.replace(",", "").astype('float')
train_prod_df.employee1 = train_prod_df.employee1.astype('float')
train_prod_df.employee2 = train_prod_df.employee2.astype('float')
train_prod_df.OC = train_prod_df.OC.astype('str').str.replace(" ", "")

학습 및 테스트 데이터셋 결합
train_prod_df 및 test_prod_df 데이터프레임이 append() 함수를 사용하여 단일 데이터프레임 train_test_prod에 결합됩니다. 이는 학습 및 테스트 데이터셋에 동시에 동일한 전처리 단계를 수행하기 위함입니다.

In [ ]:
# - Combine train and test datasets
train_test_prod = train_prod_df.append(test_prod_df)

결측값 처리
데이터셋의 결측값은 다음과 같은 방법으로 처리됩니다:

특정 병원의 잘못된 정보 수정: 'inst_id'가 430 및 413인 병원의 'instkind' 및 'bedCount'가 수정됩니다.


문자열 속성의 결측값을 'Not_sure'로 대체: 문자열 데이터 타입의 열에서 결측값은 'Not_sure' 문자열로 대체됩니다.


수치형 속성의 결측값을 -999로 대체: 수치형 데이터 타입의 열에서 결측값은 값 -999로 대체됩니다.


이는 결측된 데이터의 따로 구분해 영향을 최대한 억제하는 효과가 있습니다.

In [ ]:
# - Handle missing values
#   Correct the wrong information of specific hospitals
train_test_prod.loc[train_test_prod.inst_id == 430, ['instkind']] = 'dental_clinic'
train_test_prod.loc[train_test_prod.inst_id == 430, ['bedCount']] = 0
train_test_prod.loc[train_test_prod.inst_id == 413, ['bedCount']] = -999

#   Replace missing values in string attributes with 'Not_sure'
factor_columns = train_test_prod.select_dtypes(include=['object']).columns
train_test_prod[factor_columns] = train_test_prod[factor_columns].fillna('Not_sure')

#   Replace missing values in numeric attributes with -999
numeric_columns = train_test_prod.columns.difference(factor_columns)
train_test_prod[numeric_columns] = train_test_prod[numeric_columns].fillna(-999)

범주형 데이터를 수치형 형식으로 변환


데이터프레임의 범주형 데이터는 scikit-learn의 LabelEncoder 클래스를 사용하여 수치형 형식으로 변환됩니다. LabelEncoder는 범주형 열의 각 카테고리에 고유한 정수를 할당합니다.

In [ ]:
#   Convert categorical data to numerical format
fac_le = LabelEncoder()
train_test_prod[factor_columns] = train_test_prod[factor_columns].apply(lambda x: fac_le.fit_transform(x))

학습 및 테스트 데이터 분할


결합된 데이터프레임에서 전처리가 완료되면, 학습 및 테스트 데이터는 'OC' 열의 값에 기반하여 두 개의 데이터프레임, train_prod 및 test_prod,로 다시 분할됩니다. 그런 다음 train_prod 데이터프레임의 'OC' 열은 1 기반 인덱싱에서 0 기반 인덱싱으로 값을 변환하기 위해 1씩 감소됩니다.

In [ ]:
# - Split the data back into train and test
train_prod = train_test_prod[train_test_prod.OC != 0]
test_prod = train_test_prod[train_test_prod.OC == 0]
train_prod['OC'] = train_prod['OC'] - 1

모델 학습을 위한 데이터 준비

마지막으로, train_prod 및 test_prod 데이터프레임은 특징 변수(X)와 목표 변수(Y)로 분할됩니다. 

'OC' 열은 목표 변수이고, 나머지 모든 열은 특징 변수입니다. sub_id 변수도 생성되어 test_prod 데이터프레임에서 'inst_id' 열을 저장하며, 제출 파일에 사용됩니다.

In [ ]:
sub_id = test_prod.inst_id  # ID for submission
dep = 'OC'
indep = train_prod.columns.difference([dep])
train_prod_X = train_prod[indep]
train_prod_Y = train_prod[dep]
test_prod_X = test_prod[indep]